# Module3: 様々な環境へのデプロイ
30分

モジュール2では、いくつかのパターンでモデルの学習を実演しました。モジュール３では学習済みモデルを本番環境にデプロイする、と言うシナリオを意識します。デプロイ先にはいくつかの選択肢がありますが、[ローカル](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-local-container-notebook-vm)、[Azure Kubernetes Service](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-azure-kubernetes-service?tabs=python)、[Azure Container Instances](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-azure-container-instance)などがその一例です。
今回は Azure Container Instances(ACI)　と Azure Kubernetes Serivice(AKS) へのデプロイを実演します。


1. [ACI を使ってリアルタイム推論サービスを作成する](#ACI-を使ってリアルタイム推論サービスを作成する)
    1. [ワークスペースへの接続](#ワークスペースへの接続)
    1. [モデルをWebサービスとしてデプロイ](#モデルを-Web-サービスとしてデプロイ)
    1. [モデルを Web サービスとしてデプロイ](#モデルを-Web-サービスとしてデプロイ)
    1. [Web サービスをコール](#Web-サービスをコール)
    1. [サービスの削除](#サービスの削除)
1. [AKS を使って推論クラスタを作成](#AKS-を使って推論クラスタを作成)
    1. [推論クラスタの作成](#推論クラスタの作成)
    1. [推論クラスタにデプロイ](#推論クラスタにデプロイ)
    1. [AKS にリクエスト](#AKS-にリクエスト)
    
 

# ACI を使ってリアルタイム推論サービスを作成する

予測モデルを学習すると、クライアントから新しいデータに対する予測値を取得するようなリアルタイムサービスをデプロイすることができます。

## ワークスペースへの接続

まずはワークスペースに接続します。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## モデルを Web サービスとしてデプロイ

前のモジュールで、糖尿病かどうかの尤もらしさを基に患者を分類するモデルを学習し登録しています。医療現場の特定のシナリオにおいて、このモデルは本番環境で使えるかもしれません。本番環境を想定し、Web サービスとしてデプロイします。

まずはワークスペースにどのようなモデルが登録されているか確認しましょう。

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

それではデプロイしたいモデルを取得しましょう。モデル名をしてした際、デフォルトでは最新のバージョンが返ってきます。

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

取得したモデルをホストする Web サービスを作成します。これにはいくつかのスクリプトと設定ファイルが必要ですので、フォルダを作成し準備していきます。

In [ ]:
import os

folder_name = 'diabetes_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_diabetes.py")

モデルをデプロイする Web サービスでは、入力データの読み込み、ワークスペースからモデルの取得、そして予測値を返すための Python コードが必要です。以下のように Web サービスにデプロイされる *entry script* (あるいは *scoring script*) に保存します。

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Web サービスはコンテナの中にホストされ、コンテナは初期化の際に　Python の必要なパッケージをインストールする必要があります。今回の推論コードでは **scikit-learn** が必要で、この情報を YAML ファイルでコンテナに与えます。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"diabetes_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

以上でデプロイの準備が完了です。それでは **diabetes-service** と言うサービス名のコンテナをデプロイしましょう。デプロイのプロセスは以下の通りです。

1. 推論設定(モデルをロードし使用するための推論スクリプト、環境ファイル)を定義
2. デプロイ定義(ホストされたサービスの実行環境の定義。ここではAzure Container Instance)
3. モデルを Web サービスとしてデプロイ
4. デプロイされたサービスの状態を確認

> **詳細**: モデルのデプロイと実行環境のオプションについては [ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)をご覧ください。

コンテナイメージを作成する手続きが最初に発生するため、デプロイには少し時間がかかります。作成されたイメージを基に Web サービスが作成されます。デプロイに成功すると、ステータスが **Healthy** と表示されます。

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

何事もなく **Healthy** と表示されているといいのですが、もし失敗している場合は以下のコードでトラブルシューティングのためのログが表示できます。

In [ ]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

[Azure Machine Learning Studio](https://ml.azure.com) のワークスペースからエンドポイントのページを開き、デプロイされているサービスを表示してみましょう。

下記のコードからもワークスペース内の Web サービスを取得することができます。

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Web サービスをコール

デプロイされたサービスを、クライアントアプリケーションからコールすることができます。

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

複数行のデータを推論することも可能です。

In [ ]:
# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

上記のコードでは Azure Machine Learning SDK を使ってコンテナ化された Web サービスに接続し、学習済みモデルで予測値を生成しています。本番環境では、モデルは Azure Machine Learning SDK を使っていないビジネスアプリケーションから利用されるより、単純に HTTP リクエストで Web サービスを利用する方が一般的でしょう。

それではアプリケーションのアクセス先となる URL を定義しましょう。

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

これでエンドポイントの URL が取得できましたので、HTTP リクエストを作成し、患者データを JSON 形式で送信し、予測カテゴリを取得します。

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

これまでの処理で、認証を必要としない Web サービスを Azure Container Instance としてデプロイしました。開発・検証の環境であれば十分ですが、本番環境では Azure Kubernetes Services (AKS) クラスタへのデプロイとトークンベースの認証を検討すべきです。これは **Authorization** のヘッダーを含む　REST リクエストが要求されます。

## サービスの削除

サービスがもう必要ない状況になれば、不要な費用を避けるためにも削除しておくことを推奨します。

In [ ]:
service.delete()
print ('Service deleted.')

# AKS を使って推論クラスタを作成

最後に AKS を使った推論クラスタへのデプロイを実演します。
前述の通り、本番環境ではリクエスト数に対してスケールさせたり、認証を意識した設計が重要です。

## 推論クラスタの作成
まずはデプロイするためのクラスタを作成します。今回は Azure Machine Learning Studio から AKS を作成してみます。
コンピューティングから推論クラスタを選び、新規を選びます。

## 推論クラスタにデプロイ
AKS にデプロイするために必要なコンピューティングリソースを記述したデプロイ構成を作成します。[ドキュメント](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-azure-kubernetes-service?tabs=python#deploy-to-aks)のサンプルコードを流用しています。
デプロイの設定ではコア数、メモリ量が指定されていますが、これらは Web サービスに割り当てられるリソースを意味しています。

作成が成功したら下記の *your-compute-cluster* をクラスタ名を置き換えてください。

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute

cluster_name = "your-compute-cluster"
service_name = "aks-service"

aks_target = AksCompute(ws, cluster_name)
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())
print(service.state)

デプロイが成功していれば、**Healthy** と表示されるはずです。

## AKS にリクエスト
Azure Machine Learning Studio のエンドポイントに、デプロイされているサービスの一覧が表示されます。今回デプロイしたサービスを開くと、使用というタブから URL や認証用のキーが確認できます。以下のようなサンプルコードでアクセスすることができます。(一部修正しています)

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

#data = json.dumps({"data": x_new})
data = {"data": x_new}

body = str.encode(json.dumps(data))

url = service.scoring_uri
api_key, _ = service.get_keys()
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, whAKS にリクエストich are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

Azure Container Instance と同様の結果が得られたことが確認できました。
ACI の例と同様、サービスが不要になれば削除しておきましょう。

In [ ]:
service.delete()
print ('Service deleted.')

また、AKS 自体は削除されていないので、クラスタ自体が不要な場合は忘れずに削除しましょう。